In [1]:
from unsloth import FastLanguageModel
import torch
import json
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [19]:
#加载模型
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
inputs = tokenizer("你好，请生成一段文本。", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [5]:
alpaca_prompt = """
### Instruction:
# 角色
你是一位专业的心理学家，擅长通过分析个人的行为和言语来评估其性格特质。你的任务是根据用户提供的信息，对个体的性格进行详细分析，并给出五大性格特质的具体评分。

- **任务**：基于用户提供的信息（如行为描述、言语表达等），对个体的五大性格特质进行评估，并给出从0-1的具体分数。
  - **开放性（Openness）**：评估个体的好奇心、想象力和对新事物的接受程度。
  - **责任心（Conscientiousness）**：评估个体的责任感、组织能力和自律性。
  - **外向性（Extraversion）**：评估个体的社交能力、活力和乐观程度。
  - **宜人性（Agreeableness）**：评估个体的合作性、同情心和信任度。
  - **神经质（Neuroticism）**：评估个体的情绪稳定性、焦虑水平和压力应对能力。

## 限制
- 仅基于用户提供的信息进行评估，确保评估结果客观准确。

### Input:
{}
### Response:
{}"""
myData = json.load(open('../data/results.json', 'r',encoding='utf-8'))

In [18]:
ai_input = myData[list(myData.keys())[0]]
for i in ai_input.keys():
    # print(ai_input[i])
    print(alpaca_prompt.format(ai_input[i], ' '))
    print("-"*50)



### Instruction:
# 角色
你是一位专业的心理学家，擅长通过分析个人的行为和言语来评估其性格特质。你的任务是根据用户提供的信息，对个体的性格进行详细分析，并给出五大性格特质的具体评分。

- **任务**：基于用户提供的信息（如行为描述、言语表达等），对个体的五大性格特质进行评估，并给出从0-1的具体分数。
  - **开放性（Openness）**：评估个体的好奇心、想象力和对新事物的接受程度。
  - **责任心（Conscientiousness）**：评估个体的责任感、组织能力和自律性。
  - **外向性（Extraversion）**：评估个体的社交能力、活力和乐观程度。
  - **宜人性（Agreeableness）**：评估个体的合作性、同情心和信任度。
  - **神经质（Neuroticism）**：评估个体的情绪稳定性、焦虑水平和压力应对能力。

## 限制
- 仅基于用户提供的信息进行评估，确保评估结果客观准确。

### Input:
From the audio analysis, the speaker said: 当然可以，我曾经负责过一家本地餐饮品牌的推广活动。当时客户希望在短时间内提升品牌知名度，并增加客流量。我首先进行了市场调研，分析了客户目标群体的爱好和习惯消费习惯。然后我们设计了一套线上线下结合的。Yeah.推广方案线上部分，我们通过社交媒体平台发布创意短视频与活动交活动话题，吸引年轻消费者关注线下部分。我们在商场和学校附近举办实吃活动，并与周边商家合作，进行联合推广。最终这个活动在一个月内。将品牌的线上粉丝增加了30%，客流量提升了25%。客户对我们的工作表示很满意。.
The most possible emotion is 中立/neutral with score 1.0. 
His speech rate is 0.01689189189189189 words per second, the average volume is -10.10 dB 	 the standard deviation of the volume is 5.21 dB. The average pitch is 119.80 Hz 	 the standard deviation of the pitch is:9.

In [20]:
for i in ai_input.keys():
    prompt = alpaca_prompt.format(ai_input[i], ' ')
    prompt_encoding = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **prompt_encoding,
        use_cache=True,
        max_new_tokens=128,
        temperature=0.75,
        top_k=50,
        top_p=0.9
    )
    # 将生成的输出解码为文本
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # 打印生成的文本
    print(generated_text[0])



### Instruction:
# 角色
你是一位专业的心理学家，擅长通过分析个人的行为和言语来评估其性格特质。你的任务是根据用户提供的信息，对个体的性格进行详细分析，并给出五大性格特质的具体评分。

- **任务**：基于用户提供的信息（如行为描述、言语表达等），对个体的五大性格特质进行评估，并给出从0-1的具体分数。
  - **开放性（Openness）**：评估个体的好奇心、想象力和对新事物的接受程度。
  - **责任心（Conscientiousness）**：评估个体的责任感、组织能力和自律性。
  - **外向性（Extraversion）**：评估个体的社交能力、活力和乐观程度。
  - **宜人性（Agreeableness）**：评估个体的合作性、同情心和信任度。
  - **神经质（Neuroticism）**：评估个体的情绪稳定性、焦虑水平和压力应对能力。

## 限制
- 仅基于用户提供的信息进行评估，确保评估结果客观准确。

### Input:
From the audio analysis, the speaker said: 当然可以，我曾经负责过一家本地餐饮品牌的推广活动。当时客户希望在短时间内提升品牌知名度，并增加客流量。我首先进行了市场调研，分析了客户目标群体的爱好和习惯消费习惯。然后我们设计了一套线上线下结合的。Yeah.推广方案线上部分，我们通过社交媒体平台发布创意短视频与活动交活动话题，吸引年轻消费者关注线下部分。我们在商场和学校附近举办实吃活动，并与周边商家合作，进行联合推广。最终这个活动在一个月内。将品牌的线上粉丝增加了30%，客流量提升了25%。客户对我们的工作表示很满意。.
The most possible emotion is 中立/neutral with score 1.0. 
His speech rate is 0.01689189189189189 words per second, the average volume is -10.10 dB 	 the standard deviation of the volume is 5.21 dB. The average pitch is 119.80 Hz 	 the standard deviation of the pitch is:9.

In [ ]:
# prompt = alpaca_prompt.format(final_dataset['test'][index]['input'], ' ')
# prompt_encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
# outputs = model.generate(
#     **prompt_encoding,
#     use_cache=True,
#     max_new_tokens=128,
#     temperature=0.75,
#     top_k=50,
#     top_p=0.9
# )
# # 将生成的输出解码为文本
# generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
# 
# # 打印生成的文本
# print(generated_text[0])
# print("以上为模型输出，以下为测试集的真实输出：")
# print(final_dataset['test'][index]['output'])
# temp_list  =[]
# for i in get_scores_from_text(index).values():
#     temp_list.append(i)